### Баю-бай, поскорее засыпай: Генерим предложения по сказкам
В рамках данной домашки я попытаюсь реализовать хотя бы около осмысленные предложения по сказкам


**Датасет:** Несколько книг детских сказок на английском

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import re
import itertools
import random
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt_tab')
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Собираем тексты:

In [2]:
def download(file_name):
    with open(file_name,'r', encoding='UTF-8') as file:
        book = file.read()
        return book

In [3]:
text_0 = download('/content/book_1.txt')
text_1 = download('/content/book_2.txt')
text_2 = download('/content/book_3.txt')
text_3 = download('/content/pg4357.txt')


print(text_0[:100])

How Night Came


Years and years ago at the very beginning of time, when the world had
just been mad


### Чистим тексты

In [4]:
def split_sentences(text):
    sentences = sent_tokenize(text, language="english")
    return sentences

def clean_text(text):

    """
    Удаляем, знаки препинания, табы, абзацы, цифры, двойные пробелы. Приводим к нижнему регистру
    """
    sentences = list()
    for sentence in text:
        sentence = re.sub(r'[^\w\s]', ' ', sentence)
        sentence = re.sub(r'[0-9]','', sentence)
        sentence = sentence.lower()
        sentence = re.sub(r'\s+',' ', sentence)
        sentence = re.sub(r'глас рассудка','', sentence).strip()
        sentences.append(sentence)
    return sentences

def total_slay(text):
    sentences = split_sentences(text)
    cleaned_sentences = clean_text(sentences)
    return cleaned_sentences


In [5]:
text_1_ = total_slay(text_0)
text_2_ = total_slay(text_1)
text_3_ = total_slay(text_2)
text_4_ = total_slay(text_3)

In [6]:
data = list(itertools.chain(text_1_,text_2_,text_3_,text_4_))
data = data[:5000]

In [7]:
# Инициализируем токенизатор
tokenizer = Tokenizer(num_words=1000)

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

In [8]:
X[:10], y[:10]

([[87],
  [87, 92],
  [87, 92, 46],
  [87, 92, 46, 247],
  [87, 92, 46, 247, 2],
  [87, 92, 46, 247, 2, 247],
  [87, 92, 46, 247, 2, 247, 756],
  [87, 92, 46, 247, 2, 247, 756, 25],
  [87, 92, 46, 247, 2, 247, 756, 25, 1],
  [87, 92, 46, 247, 2, 247, 756, 25, 1, 49]],
 [92, 46, 247, 2, 247, 756, 25, 1, 49, 707])

In [9]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)
# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [ ]:
# Создаем модель
model = Sequential()


model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=X.shape[1]))


model.add(GRU( 128, return_sequences=False, kernel_regularizer=l2(0.01)))
model.add(Dropout(0,2))

model.add(Dense(len(tokenizer.word_index) + 1,activation='softmax'))

optimizer = Adam(learning_rate=0.001)  # Можно настроить learning_rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Обучаем модель
history = model.fit(X, y, epochs=25, batch_size=128, validation_split=0.2)

Epoch 1/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.0782 - loss: 6.4504 - val_accuracy: 0.0696 - val_loss: 5.7546
Epoch 2/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.0818 - loss: 5.5807 - val_accuracy: 0.0842 - val_loss: 5.5733
Epoch 3/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.0993 - loss: 5.2950 - val_accuracy: 0.0929 - val_loss: 5.5100
Epoch 4/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.1102 - loss: 5.1543 - val_accuracy: 0.0991 - val_loss: 5.4687
Epoch 5/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.1138 - loss: 5.0915 - val_accuracy: 0.1042 - val_loss: 5.4389
Epoch 6/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.1206 - loss: 4.9793 - val_accuracy: 0.1108 - val_loss: 5.3753
Epoch 7/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.1287 - loss: 4.8558 - val_accuracy: 0.1127 - val_loss: 5.3179
Epoch 8/25
494/494 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1348 - loss: 4.7394 - val

In [23]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 97, 50)              │         276,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 128)                 │          69,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5520)                │         712,080 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,171,602 (12.10 MB)

 Trainable params: 1,057,200 (4.03 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,114,402 (8.07 MB)

In [27]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый заголовок
generated_text = generate_text("king", 5, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
king is the little white hen
